#### Задание: 
Поэксперементировать с разными подходами (как я понял)

In [1]:
import numpy as np

import torch
from torch import optim
from torch import nn
from torch.nn import functional as F

import torchvision
from torchvision import models
from torchvision import transforms, datasets

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# устраняем проблему с загрузкой cifar10 (certificate verify failed: certificate has expired (_ssl.c:1123))
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
data = datasets.CIFAR10(root='./data', train=True, download=True)

Files already downloaded and verified


##### Подготовим данные

In [3]:
class MyOwnCifar(torch.utils.data.Dataset):

    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
            img = self.transform(img)
        return img, self._base_dataset[idx][1]

In [4]:
 train_transformation = transforms.Compose([transforms.Resize(256),
                                           transforms.RandomCrop(
                                               32, padding=4),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                std=[0.229, 0.224, 0.225])])
test_transformation = transforms.Compose([transforms.ToTensor(),
                                          transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                               std=[0.229, 0.224, 0.225])])

In [5]:
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

train_dataset, test_dataset = MyOwnCifar(train_data, train_transformation), MyOwnCifar(test_data, test_transformation)

BATCH_SIZE = 128

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True,
                                           num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=False,
                                          num_workers=0)



def train_proc(dataloader, model, loss_fn, optimizer):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        optimizer.zero_grad()
        
        # Compute prediction error
        y_pred = model(X)
        loss = loss_fn(y_pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()

    print(f'loss: {loss.item()}')



def test_proc(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [6]:
EPOCHS = 10
LR = 0.01

#### Попробуем использовать Resnet50 вместо Resnet18 на уроке

In [7]:
resnet_50 = models.resnet50(pretrained=True)

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose([transforms.Resize(
    256), transforms.CenterCrop(32), normalize, transforms.ToTensor()])

set_parameter_requires_grad(resnet_50, True)
resnet_50.fc = nn.Linear(2048, 100)

params_to_update = []
for name, param in resnet_50.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

print(params_to_update)

[Parameter containing:
tensor([[ 0.0203, -0.0027, -0.0053,  ...,  0.0207, -0.0099,  0.0141],
        [ 0.0204, -0.0192,  0.0027,  ..., -0.0143,  0.0116, -0.0208],
        [-0.0187, -0.0197, -0.0104,  ..., -0.0149, -0.0019,  0.0165],
        ...,
        [-0.0202,  0.0194,  0.0088,  ...,  0.0091,  0.0211, -0.0010],
        [-0.0083, -0.0212, -0.0198,  ..., -0.0108, -0.0207,  0.0026],
        [-0.0190, -0.0151, -0.0217,  ...,  0.0012, -0.0031,  0.0038]],
       requires_grad=True), Parameter containing:
tensor([-1.8219e-02, -1.7849e-02, -1.1307e-02, -1.1527e-02, -6.3691e-03,
         2.0496e-03,  1.5061e-03, -1.2131e-02, -1.7337e-03,  3.6975e-03,
        -1.9183e-03, -1.8688e-02,  1.5951e-03,  5.2839e-03, -1.8093e-02,
        -1.5752e-02,  1.5718e-02, -5.9964e-03,  5.6270e-03, -4.7985e-03,
         1.0190e-02, -8.6541e-03,  1.3448e-02, -6.2780e-03, -9.7231e-03,
        -1.6470e-03,  8.7356e-03,  4.3749e-03, -1.8033e-02,  1.6627e-02,
        -6.3104e-03, -2.5149e-03,  6.8446e-03, -2.0358e

In [8]:
optimizer = torch.optim.Adam(params_to_update, lr=LR)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    print(f'Epoch {epoch+1}\n************************')
    train_proc(train_loader, resnet_50, loss_fn, optimizer)
    test_proc(test_loader, resnet_50, loss_fn)

Epoch 1
************************
loss: 2.6652519702911377
Test Error: 
 Accuracy: 18.1%, Avg loss: 5.724757 

Epoch 2
************************
loss: 2.8209314346313477
Test Error: 
 Accuracy: 18.9%, Avg loss: 5.560383 

Epoch 3
************************
loss: 2.9962778091430664
Test Error: 
 Accuracy: 17.3%, Avg loss: 4.903577 

Epoch 4
************************
loss: 2.7185380458831787
Test Error: 
 Accuracy: 16.2%, Avg loss: 5.895540 

Epoch 5
************************
loss: 2.756601095199585
Test Error: 
 Accuracy: 14.5%, Avg loss: 6.170953 

Epoch 6
************************
loss: 3.4411797523498535
Test Error: 
 Accuracy: 15.9%, Avg loss: 5.741276 

Epoch 7
************************
loss: 3.13755464553833
Test Error: 
 Accuracy: 17.2%, Avg loss: 6.351320 

Epoch 8
************************
loss: 3.054412364959717
Test Error: 
 Accuracy: 14.2%, Avg loss: 5.411817 

Epoch 9
************************
loss: 2.7345707416534424
Test Error: 
 Accuracy: 13.6%, Avg loss: 6.454932 

Epoch 10
*****

#### AUGMENTATION

In [9]:
train_transformation = transforms.Compose([ 
                                    transforms.Resize(256),
                                    transforms.RandomCrop(32, padding=4),
                                     # Augmentation
                                    transforms.RandomHorizontalFlip(p=0.5),
                                    transforms.RandomVerticalFlip(p=0.5),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])
                                    ])
test_transformation = transforms.Compose([
                                        transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])
                                       ])

In [10]:
_train_dataset, test_dataset = MyOwnCifar(train_data, train_transformation), MyOwnCifar(test_data, test_transformation)

_train = torch.utils.data.ConcatDataset([train_dataset, _train_dataset])

_train_loader = torch.utils.data.DataLoader(_train,
                                            batch_size=BATCH_SIZE,
                                            shuffle=True,
                                            num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=False,
                                          num_workers=0)

model = models.resnet50(pretrained=True)
set_parameter_requires_grad(model, True)

model.fc = torch.nn.Linear(2048, 1024)
model.fc = nn.Sequential(
    torch.nn.Dropout(0.5),
    torch.nn.Linear(480, 1024),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(1024, 512),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(512, 256),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(256, 128),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(128, len(data.classes))
)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

In [12]:
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

print(params_to_update)

[Parameter containing:
tensor([[ 0.0139, -0.0387,  0.0179,  ...,  0.0215,  0.0442,  0.0400],
        [ 0.0409, -0.0011, -0.0409,  ..., -0.0075,  0.0160,  0.0155],
        [-0.0360, -0.0150, -0.0233,  ..., -0.0452,  0.0080, -0.0274],
        ...,
        [-0.0046, -0.0083,  0.0383,  ..., -0.0166, -0.0070,  0.0217],
        [-0.0263,  0.0281,  0.0394,  ...,  0.0148, -0.0302,  0.0237],
        [-0.0229,  0.0199,  0.0365,  ..., -0.0100, -0.0264,  0.0150]],
       requires_grad=True), Parameter containing:
tensor([-0.0078,  0.0085, -0.0141,  ...,  0.0195, -0.0296, -0.0241],
       requires_grad=True), Parameter containing:
tensor([[-0.0060,  0.0145,  0.0079,  ..., -0.0114, -0.0065, -0.0223],
        [-0.0038,  0.0286,  0.0005,  ..., -0.0281, -0.0044, -0.0028],
        [ 0.0250,  0.0151,  0.0223,  ...,  0.0237, -0.0242,  0.0170],
        ...,
        [ 0.0255,  0.0269, -0.0189,  ...,  0.0205, -0.0285, -0.0094],
        [-0.0108,  0.0105, -0.0165,  ...,  0.0189, -0.0145,  0.0081],
        [-0

In [13]:
for epoch in range(EPOCHS):
    print(f'Epoch {epoch+1}\n************************')
    train_proc(_train_loader, resnet_50, loss_fn, optimizer)
    test_proc(test_loader, resnet_50, loss_fn)

Epoch 1
************************
loss: 2.826258659362793
Test Error: 
 Accuracy: 17.6%, Avg loss: 5.220430 

Epoch 2
************************
loss: 2.4279022216796875
Test Error: 
 Accuracy: 17.4%, Avg loss: 5.260569 

Epoch 3
************************
loss: 2.9510228633880615
Test Error: 
 Accuracy: 16.9%, Avg loss: 5.444481 

Epoch 4
************************
loss: 2.6105337142944336
Test Error: 
 Accuracy: 17.4%, Avg loss: 5.282579 

Epoch 5
************************
loss: 2.749117612838745
Test Error: 
 Accuracy: 17.2%, Avg loss: 5.415616 

Epoch 6
************************
loss: 2.971696376800537
Test Error: 
 Accuracy: 17.2%, Avg loss: 5.075065 

Epoch 7
************************
loss: 2.689145088195801
Test Error: 
 Accuracy: 17.2%, Avg loss: 5.339429 

Epoch 8
************************
loss: 2.903681755065918
Test Error: 
 Accuracy: 17.5%, Avg loss: 5.305566 

Epoch 9
************************
loss: 2.3875374794006348
Test Error: 
 Accuracy: 17.4%, Avg loss: 5.437481 

Epoch 10
******

#### CNN

In [43]:
class CNNNetwork(nn.Module):

    def __init__(self):
        super(CNNNetwork, self).__init__()
        
        self.dp_three = nn.Dropout(0.2)
        self.dp_four = nn.Dropout(0.2)
        
        self.bn_one = torch.nn.BatchNorm2d(3) 
        self.conv_one = torch.nn.Conv2d(3, 30, 3)
        self.bn_two = torch.nn.BatchNorm2d(30) 
        self.conv_two = torch.nn.Conv2d(30, 30, 3)
        self.bn_three = torch.nn.BatchNorm2d(30)
        self.conv_three = torch.nn.Conv2d(30, 120, 3)
        self.bn_four = torch.nn.BatchNorm2d(120)
        
        self.fc1 = torch.nn.Linear(480, 200)
        self.fc2 = torch.nn.Linear(200, 30)
        self.out = torch.nn.Linear(30, 10)
        
        
    def forward(self, x):
        
        x = self.bn_one(x)
        x = self.conv_one(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_two(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_three(x)
        x = self.conv_three(x)
        x = F.leaky_relu(x, 0.1)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_four(x)
        x = x.view(x.size(0), -1)
        x = self.dp_three(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp_four(x)
        x = self.fc2(x)
        x = F.relu(x)

        return self.out(x)

In [44]:
cnn_net = CNNNetwork()
print(cnn_net)

CNNNetwork(
  (dp_three): Dropout(p=0.2, inplace=False)
  (dp_four): Dropout(p=0.2, inplace=False)
  (bn_one): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_one): Conv2d(3, 30, kernel_size=(3, 3), stride=(1, 1))
  (bn_two): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_two): Conv2d(30, 30, kernel_size=(3, 3), stride=(1, 1))
  (bn_three): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_three): Conv2d(30, 120, kernel_size=(3, 3), stride=(1, 1))
  (bn_four): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=480, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=30, bias=True)
  (out): Linear(in_features=30, out_features=10, bias=True)
)


In [45]:
optimizer = torch.optim.Adam(params_to_update, lr=LR)
loss_fn = nn.CrossEntropyLoss()

In [46]:
for epoch in range(EPOCHS):
    print(f'Epoch {epoch+1}\n************************')
    
    train_proc(train_loader, cnn_net, loss_fn, optimizer)
    test_proc(test_loader, cnn_net, loss_fn)

Epoch 1
************************
loss: 2.286555290222168
Test Error: 
 Accuracy: 10.6%, Avg loss: 2.323469 

Epoch 2
************************
loss: 2.307523250579834
Test Error: 
 Accuracy: 10.5%, Avg loss: 2.325544 

Epoch 3
************************
loss: 2.3169312477111816
Test Error: 
 Accuracy: 10.6%, Avg loss: 2.324357 

Epoch 4
************************
loss: 2.323059558868408
Test Error: 
 Accuracy: 10.5%, Avg loss: 2.325203 

Epoch 5
************************
loss: 2.3133397102355957
Test Error: 
 Accuracy: 10.4%, Avg loss: 2.324558 

Epoch 6
************************
loss: 2.322117567062378
Test Error: 
 Accuracy: 10.7%, Avg loss: 2.323213 

Epoch 7
************************
loss: 2.3181731700897217
Test Error: 
 Accuracy: 10.4%, Avg loss: 2.325971 

Epoch 8
************************
loss: 2.351201057434082
Test Error: 
 Accuracy: 10.5%, Avg loss: 2.323954 

Epoch 9
************************
loss: 2.318372964859009
Test Error: 
 Accuracy: 10.5%, Avg loss: 2.322660 

Epoch 10
*******